In [17]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from matplotlib import pyplot as plt
import pickle
import sys
sys.path.append('./../')
sys.path.append('./../../')
import Portfolio_value as pv
# import import_ipynb

In [18]:
df =  pd.read_csv('./data/data.csv')
folder_name = 'dyna'


In [19]:
p_val = np.load('./result/{}/p_val/p_val_0.08.npy'.format(folder_name), allow_pickle=True)
w_val = np.load('./result/{}/w_val/w_val_0.08.npy'.format(folder_name), allow_pickle=True)

p_val = p_val.reshape(1)
p_val = p_val[0]

w_val = w_val.reshape(1)
w_val = w_val[0]

In [20]:
bear_year = np.load('./result/year/bear_year.npy', allow_pickle=True).reshape(1)[0]


In [21]:
upper_years = [35]

In [22]:

# model_names = []
# for protection_level in [1.0]:
#     for multiplier in range(50,100,10):
#         model_names.append('CPPI_{}_{}'.format(int(protection_level*100), multiplier))
        

In [23]:
model_names = []
for protection_level in [1.0]:
    for multiplier in range(5,55,5):
        model_names.append('CPPI_{}_{}'.format(int(protection_level*100), multiplier))

In [24]:
# model_names = []

# for i in ['SP', 'SL']:
#     for j in ['95', '90']:
#             model_names.append(i+'_'+j)

In [25]:
bear_perform = {}
for model_name in model_names:
    bear_perform[model_name] = {
    'dep':[],
    'max':[],
    'under':[],
    'inc':[],
    'beq':[],
    'total':[]
}

In [26]:
# PI_Korea랑은 코드가 좀 다름

def get_metric_for_PI(retire_year, k, upper_year, model_name, withrate, const_or_dyna, df, init_wealth, p_val, w_val):
    # bequest
    
    p_amounts = p_val['{}_rate{}_{}'.format(model_name, withrate, const_or_dyna)][retire_year]
    bequest = p_amounts[12*upper_year]*init_wealth*(1-withrate)/100
    
    # withdraw_amount
    w_amounts = w_val['{}_rate{}_{}'.format(model_name, withrate, const_or_dyna)][retire_year]
    withdraw_amount = sum(w_amounts)
    
    # depletion time
    if np.argwhere(p_amounts == 0).size > 0 : # 고갈이 되면
        dep_time = np.argwhere(p_amounts == 0)[0][0]
    elif np.argwhere(p_amounts == 0).size == 0: # 고갈이 안되면
        dep_time = upper_year*12
        
    # max decline
    dec = np.min(p_amounts)/p_amounts[0]
    dec = dec - 1
    
    # underwater duration
    start_date = datetime(retire_year, 1, 1)
    end_date = start_date + relativedelta(years=upper_year)
    
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m')
    df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
    
    under_count = 0
    for i in range(upper_year*12):
        if p_amounts[i]*df.iloc[i]['CPI'] < p_amounts[0]*df.iloc[0]['CPI'] :
             under_count+= 1
        
    return bequest / init_wealth * 100, withdraw_amount / init_wealth * 100, dep_time, dec * 100, under_count

In [27]:
init_wealth = 500000
withdraw_rate = 0.08
k=0
const_or_dyna = 'dyna'
upper_year = 35

for model_name in model_names:
    print(model_name)
    for years in bear_year[upper_year]:
        for year in years:
            res = get_metric_for_PI(year, k, upper_year, model_name, withdraw_rate, const_or_dyna, df, init_wealth, p_val, w_val)
            if res[2] < upper_year*12 : #고갈 시점이 upper_year보다 작으면 (이미 고갈이 되면)
                bear_perform[model_name]['dep'].append(100)
            elif res[2] >= upper_year*12: # 고갈이 안되면
                bear_perform[model_name]['dep'].append(0)
            bear_perform[model_name]['max'].append(res[3])
            bear_perform[model_name]['under'].append(res[4])
            bear_perform[model_name]['inc'].append(res[1])
            bear_perform[model_name]['beq'].append(res[0])
            bear_perform[model_name]['total'].append(res[1] + res[0])


CPPI_100_5
CPPI_100_10
CPPI_100_15
CPPI_100_20
CPPI_100_25
CPPI_100_30
CPPI_100_35
CPPI_100_40
CPPI_100_45
CPPI_100_50


In [28]:
# Define all dictionaries and indices we want to calculate mean for
dicts = [bear_perform]
indices = model_names

# Use list comprehension to compute all means at once
# Store results in lists
a = [np.round(np.mean(d[i]['dep']),1) for i in indices for d in dicts]
b = [np.round(np.mean(d[i]['max']),1) for i in indices for d in dicts]
c = [np.round(np.mean(d[i]['under']),1) for i in indices for d in dicts]
d = [np.round(np.mean(d[i]['inc']),1) for i in indices for d in dicts]
e = [np.round(np.mean(d[i]['beq']),1) for i in indices for d in dicts]
f = [np.round(np.mean(d[i]['total']),1) for i in indices for d in dicts]

# If you need to access the results separately, you can do so by indexing the lists:
# For example, a1 corresponds to a_results[0], a2 corresponds to a_results[1], etc.

In [29]:

print(model_names)
print('Depletion Prob. (\%) & & ' + ' & '.join(['{}'] * len(a)).format(*a))
print('Max. decline (\%) & & ' + ' & '.join(['{}'] * len(b)).format(*b))
print('Underwater duration (month) & & ' + ' & '.join(['{}'] * len(c)).format(*c))
print('Total withdrawal (\%) & & ' + ' & '.join(['{}'] * len(d)).format(*d))
print('Bequest (\%) & & ' + ' & '.join(['{}'] * len(e)).format(*e))
print('Total withdrawal + Bequest (\%) & & ' + ' & '.join(['{}'] * len(f)).format(*f))





['CPPI_100_5', 'CPPI_100_10', 'CPPI_100_15', 'CPPI_100_20', 'CPPI_100_25', 'CPPI_100_30', 'CPPI_100_35', 'CPPI_100_40', 'CPPI_100_45', 'CPPI_100_50']
Depletion Prob. (\%) & & 100.0 & 100.0 & 90.9 & 77.3 & 68.2 & 68.2 & 63.6 & 59.1 & 59.1 & 59.1
Max. decline (\%) & & -100.0 & -100.0 & -98.5 & -95.3 & -90.0 & -88.4 & -86.4 & -84.7 & -83.3 & -82.5
Underwater duration (month) & & 398.1 & 383.2 & 356.4 & 330.4 & 279.1 & 269.5 & 268.4 & 264.2 & 259.6 & 251.0
Total withdrawal (\%) & & 106.1 & 112.6 & 116.8 & 121.2 & 125.6 & 127.3 & 130.3 & 131.6 & 132.8 & 133.6
Bequest (\%) & & 0.0 & 0.0 & 1.5 & 4.7 & 10.9 & 14.6 & 18.6 & 21.5 & 24.0 & 26.2
Total withdrawal + Bequest (\%) & & 106.1 & 112.6 & 118.3 & 126.0 & 136.5 & 141.9 & 149.0 & 153.1 & 156.8 & 159.8


In [ ]:
['CPPI_100_5', 'CPPI_100_10', 'CPPI_100_15', 'CPPI_100_20', 'CPPI_100_25', 'CPPI_100_30', 'CPPI_100_35', 'CPPI_100_40', 'CPPI_100_45']
Depletion Prob. (\%) & & 100.0 & 81.8 & 54.5 & 40.9 & 31.8 & 27.3 & 22.7 & 22.7 & 22.7
Max. decline (\%) & & -100.0 & -97.0 & -86.5 & -77.3 & -65.2 & -64.2 & -61.6 & -59.4 & -57.1
Underwater duration (month) & & 334.8 & 267.8 & 213.6 & 184.4 & 150.7 & 145.7 & 135.0 & 133.8 & 132.3
Total withdrawal (\%) & & 109.5 & 122.8 & 126.9 & 132.2 & 133.0 & 134.0 & 134.9 & 134.9 & 135.3
Bequest (\%) & & 0.0 & 3.0 & 14.1 & 26.4 & 44.8 & 52.2 & 63.4 & 70.7 & 77.3
Total withdrawal + Bequest (\%) & & 109.5 & 125.7 & 141.0 & 158.6 & 177.9 & 186.3 & 198.3 & 205.6 & 212.6

In [13]:
['CPPI_100_1', 'CPPI_100_2', 'CPPI_100_3', 'CPPI_100_4', 'CPPI_100_5', 'CPPI_100_6', 'CPPI_100_7', 'CPPI_100_8', 'CPPI_100_9', 'CPPI_100_10', 'CPPI_100_11', 'CPPI_100_12', 'CPPI_100_13', 'CPPI_100_14', 'CPPI_100_15', 'CPPI_100_16', 'CPPI_100_17', 'CPPI_100_18', 'CPPI_100_19', 'CPPI_100_20']
Depletion Prob. (\%) & & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 95.5 & 95.5 & 81.8 & 81.8 & 72.7 & 68.2 & 54.5 & 54.5 & 54.5 & 54.5 & 54.5 & 54.5 & 50.0 & 40.9
Max. decline (\%) & & -100.0 & -100.0 & -100.0 & -100.0 & -100.0 & -100.0 & -99.7 & -99.4 & -98.4 & -97.0 & -95.8 & -93.2 & -90.3 & -88.0 & -86.5 & -85.6 & -85.6 & -84.3 & -80.8 & -77.3
Underwater duration (month) & & 368.5 & 360.5 & 349.9 & 343.1 & 334.8 & 315.5 & 297.3 & 285.6 & 276.1 & 267.8 & 253.5 & 239.5 & 223.8 & 214.6 & 213.6 & 212.8 & 210.3 & 207.7 & 197.4 & 184.4
Total withdrawal (\%) & & 101.9 & 103.7 & 105.9 & 107.4 & 109.5 & 111.8 & 115.4 & 118.9 & 121.3 & 122.8 & 123.9 & 124.4 & 125.3 & 126.0 & 126.9 & 127.6 & 128.1 & 128.4 & 130.7 & 132.2
Bequest (\%) & & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.3 & 0.6 & 1.6 & 3.0 & 4.3 & 7.0 & 10.0 & 12.5 & 14.1 & 15.1 & 15.2 & 16.7 & 20.7 & 26.4
Total withdrawal + Bequest (\%) & & 101.9 & 103.7 & 105.9 & 107.4 & 109.5 & 111.8 & 115.7 & 119.4 & 122.9 & 125.7 & 128.2 & 131.4 & 135.3 & 138.5 & 141.0 & 142.7 & 143.3 & 145.1 & 151.5 & 158.6

SyntaxError: invalid syntax (<ipython-input-13-2d141cce1759>, line 2)

In [ ]:
['CPPI_95_1', 'CPPI_95_2', 'CPPI_95_3', 'CPPI_95_4', 'CPPI_95_5', 'CPPI_95_6', 'CPPI_95_7', 'CPPI_95_8', 'CPPI_95_9', 'CPPI_95_10', 'CPPI_95_11', 'CPPI_95_12', 'CPPI_95_13', 'CPPI_95_14', 'CPPI_95_15', 'CPPI_95_16', 'CPPI_95_17', 'CPPI_95_18', 'CPPI_95_19', 'CPPI_95_20']
Depletion Prob. (\%) & & 100.0 & 95.5 & 95.5 & 86.4 & 77.3 & 63.6 & 40.9 & 40.9 & 40.9 & 36.4 & 36.4 & 40.9 & 40.9 & 45.5 & 54.5 & 54.5 & 54.5 & 54.5 & 54.5 & 54.5
Max. decline (\%) & & -100.0 & -99.8 & -99.1 & -98.2 & -93.4 & -86.8 & -80.9 & -76.4 & -73.7 & -72.0 & -71.5 & -72.8 & -75.0 & -76.8 & -77.4 & -77.6 & -77.8 & -77.8 & -77.8 & -77.4
Underwater duration (month) & & 367.0 & 351.2 & 332.7 & 309.7 & 257.0 & 229.9 & 213.6 & 189.2 & 179.1 & 170.1 & 168.3 & 173.0 & 189.4 & 195.5 & 198.9 & 200.5 & 201.7 & 203.4 & 202.9 & 200.4
Total withdrawal (\%) & & 104.4 & 109.7 & 114.5 & 120.6 & 126.9 & 132.4 & 133.7 & 133.8 & 134.1 & 134.6 & 135.3 & 133.7 & 131.9 & 129.6 & 126.7 & 125.6 & 124.0 & 123.4 & 123.1 & 123.5
Bequest (\%) & & 0.0 & 0.2 & 0.9 & 1.8 & 6.4 & 13.9 & 24.5 & 33.0 & 39.3 & 46.3 & 50.1 & 48.0 & 45.0 & 43.4 & 42.6 & 42.8 & 42.6 & 42.9 & 43.5 & 44.9
Total withdrawal + Bequest (\%) & & 104.4 & 109.9 & 115.3 & 122.3 & 133.3 & 146.3 & 158.2 & 166.8 & 173.4 & 180.8 & 185.4 & 181.7 & 177.0 & 173.0 & 169.3 & 168.4 & 166.6 & 166.3 & 166.6 & 168.4

In [ ]:
['CPPI_90_1', 'CPPI_90_2', 'CPPI_90_3', 'CPPI_90_4', 'CPPI_90_5', 'CPPI_90_6', 'CPPI_90_7', 'CPPI_90_8', 'CPPI_90_9', 'CPPI_90_10', 'CPPI_90_11', 'CPPI_90_12', 'CPPI_90_13', 'CPPI_90_14', 'CPPI_90_15', 'CPPI_90_16', 'CPPI_90_17', 'CPPI_90_18', 'CPPI_90_19', 'CPPI_90_20']
Depletion Prob. (\%) & & 100.0 & 95.5 & 81.8 & 68.2 & 36.4 & 45.5 & 45.5 & 59.1 & 63.6 & 68.2 & 63.6 & 63.6 & 63.6 & 63.6 & 63.6 & 63.6 & 63.6 & 63.6 & 63.6 & 63.6
Max. decline (\%) & & -100.0 & -98.8 & -95.4 & -87.3 & -78.5 & -77.5 & -77.7 & -80.9 & -82.6 & -83.3 & -83.2 & -82.6 & -82.0 & -81.2 & -80.7 & -80.2 & -79.9 & -79.9 & -80.0 & -80.0
Underwater duration (month) & & 364.4 & 339.6 & 302.5 & 264.2 & 216.7 & 218.6 & 220.4 & 247.9 & 264.0 & 271.0 & 274.1 & 274.4 & 272.0 & 271.2 & 270.4 & 269.2 & 268.1 & 267.5 & 267.1 & 266.6
Total withdrawal (\%) & & 108.0 & 116.6 & 126.1 & 133.3 & 135.2 & 131.0 & 128.1 & 124.2 & 118.6 & 115.6 & 113.4 & 111.6 & 111.2 & 111.2 & 111.0 & 111.2 & 111.2 & 111.5 & 111.7 & 112.1
Bequest (\%) & & 0.0 & 1.2 & 4.5 & 14.4 & 33.2 & 41.2 & 47.0 & 45.7 & 44.4 & 42.3 & 42.7 & 44.2 & 46.2 & 49.2 & 51.3 & 53.9 & 55.3 & 55.7 & 56.0 & 56.7
Total withdrawal + Bequest (\%) & & 108.0 & 117.8 & 130.6 & 147.7 & 168.4 & 172.2 & 175.1 & 169.9 & 163.0 & 157.9 & 156.1 & 155.8 & 157.5 & 160.4 & 162.3 & 165.1 & 166.4 & 167.3 & 167.8 & 168.8

In [ ]:
['CPPI_85_1', 'CPPI_85_2', 'CPPI_85_3', 'CPPI_85_4', 'CPPI_85_5', 'CPPI_85_6', 'CPPI_85_7', 'CPPI_85_8', 'CPPI_85_9', 'CPPI_85_10', 'CPPI_85_11', 'CPPI_85_12', 'CPPI_85_13', 'CPPI_85_14', 'CPPI_85_15', 'CPPI_85_16', 'CPPI_85_17', 'CPPI_85_18', 'CPPI_85_19', 'CPPI_85_20']
Depletion Prob. (\%) & & 95.5 & 86.4 & 59.1 & 31.8 & 40.9 & 59.1 & 68.2 & 63.6 & 63.6 & 63.6 & 68.2 & 63.6 & 68.2 & 68.2 & 72.7 & 72.7 & 72.7 & 68.2 & 68.2 & 68.2
Max. decline (\%) & & -99.6 & -96.5 & -84.5 & -75.1 & -75.3 & -80.3 & -83.2 & -82.7 & -82.1 & -81.7 & -81.8 & -81.7 & -81.7 & -81.6 & -81.7 & -81.7 & -81.7 & -81.6 & -81.1 & -80.5
Underwater duration (month) & & 359.7 & 324.0 & 260.0 & 211.5 & 220.5 & 255.6 & 283.7 & 284.7 & 282.1 & 282.7 & 286.4 & 286.2 & 286.8 & 289.3 & 293.0 & 295.6 & 296.9 & 297.5 & 296.8 & 291.9
Total withdrawal (\%) & & 110.8 & 124.4 & 135.2 & 135.3 & 130.9 & 120.5 & 114.2 & 113.3 & 112.0 & 112.6 & 113.0 & 112.6 & 112.6 & 111.7 & 111.8 & 110.5 & 111.1 & 111.4 & 111.6 & 112.4
Bequest (\%) & & 0.4 & 3.5 & 17.9 & 47.1 & 59.1 & 56.1 & 53.3 & 56.8 & 57.9 & 58.4 & 60.0 & 61.1 & 61.4 & 61.3 & 60.6 & 60.2 & 60.4 & 60.9 & 62.1 & 63.4
Total withdrawal + Bequest (\%) & & 111.2 & 128.0 & 153.0 & 182.4 & 190.0 & 176.6 & 167.5 & 170.1 & 169.9 & 171.0 & 173.0 & 173.7 & 174.0 & 173.0 & 172.4 & 170.8 & 171.5 & 172.3 & 173.7 & 175.8

In [ ]:
['CPPI_80_1', 'CPPI_80_2', 'CPPI_80_3', 'CPPI_80_4', 'CPPI_80_5', 'CPPI_80_6', 'CPPI_80_7', 'CPPI_80_8', 'CPPI_80_9', 'CPPI_80_10', 'CPPI_80_11', 'CPPI_80_12', 'CPPI_80_13', 'CPPI_80_14', 'CPPI_80_15', 'CPPI_80_16', 'CPPI_80_17', 'CPPI_80_18', 'CPPI_80_19', 'CPPI_80_20']
Depletion Prob. (\%) & & 95.5 & 77.3 & 50.0 & 40.9 & 59.1 & 63.6 & 59.1 & 59.1 & 59.1 & 59.1 & 59.1 & 59.1 & 59.1 & 59.1 & 59.1 & 59.1 & 63.6 & 63.6 & 63.6 & 63.6
Max. decline (\%) & & -99.1 & -91.0 & -73.8 & -74.3 & -80.1 & -79.7 & -78.0 & -77.3 & -77.6 & -77.8 & -77.5 & -77.1 & -76.8 & -76.6 & -76.6 & -76.7 & -76.9 & -76.9 & -76.8 & -76.7
Underwater duration (month) & & 352.0 & 290.0 & 210.3 & 213.4 & 271.8 & 279.0 & 271.0 & 264.5 & 268.4 & 269.9 & 267.3 & 265.4 & 263.3 & 261.4 & 260.4 & 261.0 & 263.8 & 266.4 & 267.4 & 267.0
Total withdrawal (\%) & & 115.2 & 131.5 & 136.8 & 131.4 & 121.4 & 119.9 & 119.0 & 118.7 & 117.4 & 116.3 & 116.1 & 116.1 & 115.4 & 116.1 & 116.6 & 116.8 & 116.2 & 115.5 & 114.0 & 114.5
Bequest (\%) & & 0.9 & 9.7 & 48.8 & 70.8 & 64.0 & 68.7 & 75.8 & 79.9 & 79.4 & 80.2 & 83.6 & 87.6 & 90.4 & 93.1 & 95.4 & 96.8 & 96.9 & 96.4 & 94.9 & 95.6
Total withdrawal + Bequest (\%) & & 116.2 & 141.2 & 185.6 & 202.2 & 185.4 & 188.6 & 194.8 & 198.5 & 196.8 & 196.5 & 199.7 & 203.7 & 205.8 & 209.2 & 212.0 & 213.6 & 213.1 & 211.9 & 208.9 & 210.1

In [ ]:
['CPPI_75_1', 'CPPI_75_2', 'CPPI_75_3', 'CPPI_75_4', 'CPPI_75_5', 'CPPI_75_6', 'CPPI_75_7', 'CPPI_75_8', 'CPPI_75_9', 'CPPI_75_10', 'CPPI_75_11', 'CPPI_75_12', 'CPPI_75_13', 'CPPI_75_14', 'CPPI_75_15', 'CPPI_75_16', 'CPPI_75_17', 'CPPI_75_18', 'CPPI_75_19', 'CPPI_75_20']
Depletion Prob. (\%) & & 95.5 & 54.5 & 40.9 & 54.5 & 59.1 & 54.5 & 50.0 & 50.0 & 50.0 & 50.0 & 50.0 & 50.0 & 50.0 & 50.0 & 50.0 & 45.5 & 45.5 & 45.5 & 45.5 & 45.5
Max. decline (\%) & & -98.5 & -81.5 & -70.6 & -75.8 & -75.6 & -75.2 & -74.1 & -72.4 & -72.5 & -72.7 & -73.0 & -73.9 & -74.4 & -74.4 & -74.2 & -74.0 & -73.7 & -73.4 & -73.2 & -73.1
Underwater duration (month) & & 345.3 & 246.7 & 199.5 & 245.0 & 249.6 & 250.2 & 244.1 & 237.0 & 236.4 & 237.1 & 239.5 & 245.1 & 248.2 & 247.5 & 245.9 & 242.8 & 240.3 & 238.1 & 235.4 & 234.5
Total withdrawal (\%) & & 118.4 & 135.2 & 135.4 & 124.6 & 123.6 & 122.6 & 122.3 & 122.5 & 121.6 & 121.2 & 120.5 & 119.9 & 119.9 & 120.1 & 120.1 & 120.6 & 120.6 & 120.8 & 120.9 & 120.9
Bequest (\%) & & 1.5 & 23.4 & 81.0 & 83.7 & 99.2 & 107.1 & 118.1 & 127.0 & 128.5 & 127.5 & 125.3 & 122.6 & 121.8 & 122.7 & 124.2 & 126.0 & 128.0 & 129.7 & 131.3 & 132.0
Total withdrawal + Bequest (\%) & & 119.9 & 158.6 & 216.5 & 208.3 & 222.8 & 229.7 & 240.4 & 249.5 & 250.0 & 248.7 & 245.8 & 242.6 & 241.7 & 242.8 & 244.3 & 246.6 & 248.6 & 250.5 & 252.2 & 252.9

In [ ]:
['CPPI_70_1', 'CPPI_70_2', 'CPPI_70_3', 'CPPI_70_4', 'CPPI_70_5', 'CPPI_70_6', 'CPPI_70_7', 'CPPI_70_8', 'CPPI_70_9', 'CPPI_70_10', 'CPPI_70_11', 'CPPI_70_12', 'CPPI_70_13', 'CPPI_70_14', 'CPPI_70_15', 'CPPI_70_16', 'CPPI_70_17', 'CPPI_70_18', 'CPPI_70_19', 'CPPI_70_20']
Depletion Prob. (\%) & & 86.4 & 45.5 & 45.5 & 54.5 & 45.5 & 45.5 & 45.5 & 45.5 & 45.5 & 45.5 & 40.9 & 40.9 & 40.9 & 45.5 & 45.5 & 45.5 & 45.5 & 45.5 & 50.0 & 50.0
Max. decline (\%) & & -97.6 & -74.0 & -72.8 & -74.3 & -72.3 & -71.4 & -72.5 & -72.2 & -72.2 & -72.3 & -72.3 & -72.3 & -72.2 & -72.1 & -72.0 & -71.9 & -71.8 & -71.9 & -71.9 & -71.9
Underwater duration (month) & & 339.0 & 222.9 & 216.2 & 240.4 & 230.8 & 228.7 & 235.2 & 232.8 & 231.0 & 230.2 & 229.4 & 228.3 & 226.7 & 225.1 & 223.6 & 221.2 & 219.0 & 219.1 & 219.1 & 219.2
Total withdrawal (\%) & & 123.2 & 136.1 & 132.1 & 125.6 & 123.9 & 123.9 & 123.0 & 123.7 & 124.0 & 124.3 & 124.5 & 125.5 & 126.0 & 126.3 & 127.0 & 127.4 & 127.6 & 127.6 & 127.4 & 127.4
Bequest (\%) & & 2.5 & 45.1 & 100.4 & 112.6 & 129.3 & 135.1 & 131.8 & 134.9 & 136.2 & 136.9 & 137.4 & 137.9 & 138.7 & 139.6 & 140.5 & 141.4 & 142.2 & 142.0 & 141.8 & 141.6
Total withdrawal + Bequest (\%) & & 125.7 & 181.2 & 232.5 & 238.2 & 253.3 & 259.0 & 254.7 & 258.5 & 260.2 & 261.3 & 261.9 & 263.4 & 264.7 & 265.9 & 267.5 & 268.8 & 269.8 & 269.5 & 269.2 & 269.0

In [ ]:
['CPPI_65_1', 'CPPI_65_2', 'CPPI_65_3', 'CPPI_65_4', 'CPPI_65_5', 'CPPI_65_6', 'CPPI_65_7', 'CPPI_65_8', 'CPPI_65_9', 'CPPI_65_10', 'CPPI_65_11', 'CPPI_65_12', 'CPPI_65_13', 'CPPI_65_14', 'CPPI_65_15', 'CPPI_65_16', 'CPPI_65_17', 'CPPI_65_18', 'CPPI_65_19', 'CPPI_65_20']
Depletion Prob. (\%) & & 77.3 & 40.9 & 54.5 & 50.0 & 45.5 & 40.9 & 45.5 & 45.5 & 45.5 & 45.5 & 45.5 & 45.5 & 45.5 & 50.0 & 50.0 & 50.0 & 45.5 & 45.5 & 45.5 & 45.5
Max. decline (\%) & & -95.1 & -70.2 & -73.6 & -72.0 & -72.4 & -71.9 & -71.8 & -71.6 & -71.6 & -71.6 & -71.6 & -71.7 & -71.7 & -71.9 & -71.9 & -71.9 & -71.6 & -71.2 & -70.7 & -70.3
Underwater duration (month) & & 323.2 & 206.6 & 233.3 & 228.0 & 229.9 & 224.2 & 220.5 & 214.3 & 214.3 & 214.3 & 214.4 & 214.9 & 216.4 & 217.9 & 219.3 & 218.6 & 214.2 & 211.7 & 210.0 & 208.2
Total withdrawal (\%) & & 126.6 & 136.2 & 127.0 & 125.3 & 125.8 & 126.9 & 127.4 & 128.1 & 128.1 & 128.1 & 127.9 & 127.8 & 127.6 & 127.6 & 127.4 & 127.4 & 128.1 & 128.3 & 128.7 & 129.0
Bequest (\%) & & 5.0 & 72.2 & 113.4 & 134.5 & 136.4 & 140.6 & 142.0 & 143.2 & 143.2 & 143.4 & 143.4 & 143.4 & 143.3 & 142.9 & 142.5 & 142.8 & 144.1 & 145.6 & 147.1 & 148.8
Total withdrawal + Bequest (\%) & & 131.6 & 208.4 & 240.4 & 259.8 & 262.2 & 267.5 & 269.4 & 271.4 & 271.4 & 271.5 & 271.4 & 271.2 & 270.9 & 270.5 & 269.9 & 270.2 & 272.2 & 273.9 & 275.8 & 277.8

In [ ]:
['CPPI_60_1', 'CPPI_60_2', 'CPPI_60_3', 'CPPI_60_4', 'CPPI_60_5', 'CPPI_60_6', 'CPPI_60_7', 'CPPI_60_8', 'CPPI_60_9', 'CPPI_60_10', 'CPPI_60_11', 'CPPI_60_12', 'CPPI_60_13', 'CPPI_60_14', 'CPPI_60_15', 'CPPI_60_16', 'CPPI_60_17', 'CPPI_60_18', 'CPPI_60_19', 'CPPI_60_20']
Depletion Prob. (\%) & & 72.7 & 36.4 & 50.0 & 45.5 & 40.9 & 45.5 & 45.5 & 45.5 & 45.5 & 45.5 & 45.5 & 45.5 & 45.5 & 45.5 & 45.5 & 45.5 & 45.5 & 45.5 & 45.5 & 45.5
Max. decline (\%) & & -91.5 & -69.3 & -72.0 & -71.8 & -71.0 & -70.6 & -70.1 & -70.3 & -70.4 & -70.5 & -70.5 & -70.5 & -70.5 & -70.5 & -70.4 & -70.4 & -70.4 & -70.4 & -70.3 & -70.3
Underwater duration (month) & & 294.9 & 200.3 & 230.0 & 222.5 & 211.2 & 210.5 & 208.1 & 208.5 & 208.8 & 209.3 & 209.7 & 209.3 & 209.3 & 209.2 & 209.1 & 208.9 & 208.7 & 208.5 & 208.3 & 208.1
Total withdrawal (\%) & & 130.0 & 135.9 & 126.6 & 124.0 & 125.1 & 124.4 & 124.5 & 123.6 & 123.6 & 123.1 & 123.5 & 123.5 & 123.5 & 123.5 & 123.5 & 123.5 & 123.7 & 123.7 & 123.7 & 124.1
Bequest (\%) & & 9.0 & 103.1 & 136.2 & 139.9 & 143.1 & 142.0 & 141.4 & 141.1 & 141.8 & 142.8 & 143.6 & 144.2 & 144.6 & 145.1 & 145.6 & 146.2 & 146.8 & 147.3 & 147.9 & 148.6
Total withdrawal + Bequest (\%) & & 138.9 & 239.0 & 262.8 & 263.9 & 268.1 & 266.4 & 266.0 & 264.7 & 265.4 & 265.9 & 267.0 & 267.7 & 268.1 & 268.5 & 269.1 & 269.7 & 270.4 & 271.0 & 271.6 & 272.7

In [ ]:
['CPPI_55_1', 'CPPI_55_2', 'CPPI_55_3', 'CPPI_55_4', 'CPPI_55_5', 'CPPI_55_6', 'CPPI_55_7', 'CPPI_55_8', 'CPPI_55_9', 'CPPI_55_10', 'CPPI_55_11', 'CPPI_55_12', 'CPPI_55_13', 'CPPI_55_14', 'CPPI_55_15', 'CPPI_55_16', 'CPPI_55_17', 'CPPI_55_18', 'CPPI_55_19', 'CPPI_55_20']
Depletion Prob. (\%) & & 63.6 & 36.4 & 45.5 & 45.5 & 45.5 & 45.5 & 45.5 & 40.9 & 40.9 & 40.9 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4
Max. decline (\%) & & -87.2 & -69.0 & -71.5 & -70.0 & -69.8 & -70.1 & -70.1 & -68.9 & -68.6 & -68.6 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5
Underwater duration (month) & & 280.9 & 201.7 & 221.6 & 206.6 & 204.5 & 205.9 & 205.9 & 205.9 & 200.3 & 199.3 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8
Total withdrawal (\%) & & 132.3 & 133.5 & 126.3 & 126.5 & 127.1 & 127.3 & 128.3 & 129.7 & 130.3 & 131.5 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2
Bequest (\%) & & 14.9 & 129.2 & 148.9 & 155.4 & 160.9 & 164.5 & 169.1 & 175.3 & 182.8 & 190.3 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9
Total withdrawal + Bequest (\%) & & 147.2 & 262.7 & 275.2 & 281.9 & 288.0 & 291.8 & 297.4 & 305.0 & 313.0 & 321.8 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1

In [ ]:
['CPPI_50_1', 'CPPI_50_2', 'CPPI_50_3', 'CPPI_50_4', 'CPPI_50_5', 'CPPI_50_6', 'CPPI_50_7', 'CPPI_50_8', 'CPPI_50_9', 'CPPI_50_10', 'CPPI_50_11', 'CPPI_50_12', 'CPPI_50_13', 'CPPI_50_14', 'CPPI_50_15', 'CPPI_50_16', 'CPPI_50_17', 'CPPI_50_18', 'CPPI_50_19', 'CPPI_50_20']
Depletion Prob. (\%) & & 59.1 & 40.9 & 40.9 & 40.9 & 40.9 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4
Max. decline (\%) & & -82.4 & -71.1 & -70.1 & -69.2 & -68.6 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5
Underwater duration (month) & & 261.2 & 219.9 & 206.5 & 205.7 & 199.6 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8
Total withdrawal (\%) & & 133.8 & 129.9 & 128.8 & 129.5 & 130.6 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2
Bequest (\%) & & 22.4 & 138.9 & 162.6 & 172.6 & 185.5 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9
Total withdrawal + Bequest (\%) & & 156.2 & 268.8 & 291.4 & 302.2 & 316.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1

In [ ]:
['CPPI_45_1', 'CPPI_45_2', 'CPPI_45_3', 'CPPI_45_4', 'CPPI_45_5', 'CPPI_45_6', 'CPPI_45_7', 'CPPI_45_8', 'CPPI_45_9', 'CPPI_45_10', 'CPPI_45_11', 'CPPI_45_12', 'CPPI_45_13', 'CPPI_45_14', 'CPPI_45_15', 'CPPI_45_16', 'CPPI_45_17', 'CPPI_45_18', 'CPPI_45_19', 'CPPI_45_20']
Depletion Prob. (\%) & & 50.0 & 40.9 & 40.9 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4
Max. decline (\%) & & -77.3 & -70.0 & -68.7 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5
Underwater duration (month) & & 244.5 & 218.4 & 202.9 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8
Total withdrawal (\%) & & 135.5 & 128.8 & 130.1 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2
Bequest (\%) & & 31.6 & 159.7 & 178.2 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9
Total withdrawal + Bequest (\%) & & 167.1 & 288.5 & 308.3 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1

In [ ]:
['CPPI_40_1', 'CPPI_40_2', 'CPPI_40_3', 'CPPI_40_4', 'CPPI_40_5', 'CPPI_40_6', 'CPPI_40_7', 'CPPI_40_8', 'CPPI_40_9', 'CPPI_40_10', 'CPPI_40_11', 'CPPI_40_12', 'CPPI_40_13', 'CPPI_40_14', 'CPPI_40_15', 'CPPI_40_16', 'CPPI_40_17', 'CPPI_40_18', 'CPPI_40_19', 'CPPI_40_20']
Depletion Prob. (\%) & & 45.5 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4
Max. decline (\%) & & -74.6 & -69.1 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5
Underwater duration (month) & & 228.6 & 203.5 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8
Total withdrawal (\%) & & 136.3 & 129.5 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2
Bequest (\%) & & 42.7 & 172.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9
Total withdrawal + Bequest (\%) & & 179.0 & 302.4 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1